In [1]:
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from  services.influencers_service import  InfluencersService
from shared.mongo import MongoConnection
#visualisation 
import pyLDAvis
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora
import json 
import glob
import nltk
from nltk.corpus import stopwords

2024-05-29 12:12:29,620 - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-05-29 12:12:29,621 - INFO - NumExpr defaulting to 8 threads.


In [2]:
mongo_connection=MongoConnection()
influencers_service = InfluencersService(mongo_connection)
bios=[]
posts=influencers_service.get_posts()
influencers=influencers_service.get_influencers()
for influencer in influencers:
    info_string=""
    if influencer['Bio']!= 'NULL':
        info_string+=influencer['Bio']
    if influencer['Category']!= 'NULL':
        info_string+=influencer['Category']
    if info_string!= "":
        bios.append(influencer['Bio']+" "+influencer['Category'])

2024-05-29 12:12:30,843 - INFO - MongoDB connection established successfully
2024-05-29 12:12:31,194 - INFO - Fetched 20316 posts
2024-05-29 12:12:31,369 - INFO - Fetched 38112 influencers


In [3]:
def merge_content_posts_bios(posts,bios) : 
    #concatenation of captions of all posts + bios of all influencers
    merged_content = ""
    for post in posts:
         #extract caption text
         caption = post.get('caption')
         if (caption):
            merged_content += caption + " "
    for bio in bios:
        merged_content+=bio
    return merged_content

In [4]:
def gen_words(text):
    words=[]
    new = gensim.utils.simple_preprocess(text, deacc=True)    
    words.append(new)
    return words

In [5]:
def gen_bow(words):
    id2word = corpora.Dictionary(words)
    corpus=[]
    for word in words: 
        new = id2word.doc2bow(word)
        corpus.append(new)
    return  corpus

In [6]:
def make_bigrams(texts,bigram):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts,bigram,trigram):
    return [trigram[bigram[doc]] for doc in texts]

In [7]:
# posts=posts[0:3] #for rapid testing we only use three posts
merge=merge_content_posts_bios(posts,bios)
# print(merge)
words=gen_words(merge)
# print(words)

##generate bigrams and tigrams##
bigram_phrases = gensim.models.Phrases(words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[words], threshold=100)
bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)
data_bigrams = make_bigrams(words,bigram)
data_bigrams_trigrams = make_trigrams(data_bigrams,bigram,trigram)


2024-05-29 12:12:33,110 - INFO - collecting all words and their counts
2024-05-29 12:12:33,110 - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-05-29 12:12:33,556 - INFO - collected 378929 token types (unigram + bigrams) from a corpus of 463576 words and 1 sentences
2024-05-29 12:12:33,557 - INFO - merged Phrases<378929 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-05-29 12:12:33,557 - INFO - Phrases lifecycle event {'msg': 'built Phrases<378929 vocab, min_count=5, threshold=100, max_vocab_size=40000000> in 0.45s', 'datetime': '2024-05-29T12:12:33.557267', 'gensim': '4.3.2', 'python': '3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-05-29 12:12:33,558 - INFO - collecting all words and their counts
2024-05-29 12:12:34,172 - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-05-29 12:12:34,629 - INFO - collected 383215 tok

In [8]:
from gensim.parsing.preprocessing import STOPWORDS
texts = data_bigrams_trigrams #for easier typing
custom_stopwords = {
    'com','gmail','creator','love','new','creators','dm','www','hotmail','dm','info','null','email','collab','contact','lover','celebrities','life','live','influencer',
}  # Add any additional stop words to this set
stopwords = STOPWORDS.union(custom_stopwords)
filtered_texts = []
for text in texts:
    filtered_text = [word for word in text if word not in stopwords]
    filtered_texts.append(filtered_text)
texts=filtered_texts
# print(texts)

In [9]:
# # #TF-IDF REMOVAL
# from gensim.models import TfidfModel
# # 
id2word = corpora.Dictionary(texts)
# 
corpus = [id2word.doc2bow(text) for text in texts]
# tfidf = TfidfModel(corpus, id2word=id2word,smartirs='ntc')
# 
# low_value =0.0005
# words  = []
# words_missing_in_tfidf = []
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

# print(corpus)

2024-05-29 12:12:36,421 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2024-05-29 12:12:36,710 - INFO - built Dictionary<75047 unique tokens: ['a_', 'a_lops_life', 'a_mayesing', 'a_mexicanpup', 'a_texanpup']...> from 1 documents (total 342437 corpus positions)
2024-05-29 12:12:36,711 - INFO - Dictionary lifecycle event {'msg': "built Dictionary<75047 unique tokens: ['a_', 'a_lops_life', 'a_mayesing', 'a_mexicanpup', 'a_texanpup']...> from 1 documents (total 342437 corpus positions)", 'datetime': '2024-05-29T12:12:36.711551', 'gensim': '4.3.2', 'python': '3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [28]:
lda_model=gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=4,
    random_state=50,
    chunksize=1000,
    update_every=1,
    iterations=6,
    passes=1,
    alpha="auto",
    eta="auto",
)

2024-05-29 13:45:40,919 - INFO - using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2024-05-29 13:45:40,927 - INFO - using serial LDA version on this node
2024-05-29 13:45:40,959 - INFO - running online (single-pass) LDA training, 4 topics, 1 passes over the supplied corpus of 1 documents, updating model once every 1 documents, evaluating perplexity every 1 documents, iterating 6x with a convergence threshold of 0.001000
2024-05-29 13:45:40,973 - WARNING - too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2024-05-29 13:45:41,224 - INFO - -12.168 per-word bound, 4603.0 perplexity estimate based on a held-out corpus of 1 documents with 342437 words
2024-05-29 13:45:41,226 - INFO - PROGRESS: pass 0, at document #1/1
2024-05-29 13:45:41,263 - INFO - optimized alpha [0.4666872, 0.45600092, 0.45693246, 0.47100177]
2024-05-29 13:45:41,323 - INFO - topic #0 (0.467): 0.008*"fashion" + 0.008*"lifestyle" + 0.008*"bl

In [23]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30,lambda_step=0.01)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.001584  0.003058       1        1  53.531124
2      0.001747 -0.002537       2        1  26.633827
1     -0.003330 -0.000521       3        1  19.835050, topic_info=                                     Term         Freq        Total Category  \
68591                              travel  2888.000000  2888.000000  Default   
36936                           lifestyle  3043.000000  3043.000000  Default   
6784                              blogger  2815.000000  2815.000000  Default   
50798  personal_goods_general_merchandise  1195.000000  1195.000000  Default   
63730                               style  1053.000000  1053.000000  Default   
...                                   ...          ...          ...      ...   
39472                                mama   133.599495   853.253576   Topic3   
22275                             fitness   129.061570   805.469698   Topic3   
42495                               model   139.896714  1077.874798   Topic3   
28028                                home   129.798460   854.416101   Topic3   
24683                                girl   128.221625   882.815967   Topic3   

       logprob  loglift  
68591  30.0000  30.0000  
36936  29.0000  29.0000  
6784   28.0000  28.0000  
50798  27.0000  27.0000  
63730  26.0000  26.0000  
...        ...      ...  
39472  -6.2313  -0.2365  
22275  -6.2658  -0.2134  
42495  -6.1852  -0.4241  
28028  -6.2601  -0.2667  
24683  -6.2724  -0.3116  

[276 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
291        1  0.472784   account
291        2  0.375340   account
291        3  0.147971   account
436        1  0.447722   actress
436        2  0.286542   actress
...      ...       ...       ...
74397      2  0.215555   youtube
74397      3  0.171356   youtube
74410      1  0.479967  youtuber
74410      2  0.278691  youtuber
74410      3  0.241919  youtuber

[554 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [29]:
coherence_model =gensim.models.CoherenceModel(model=lda_model, texts=texts,dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(coherence_score)

2024-05-29 13:45:49,130 - INFO - using ParallelWordOccurrenceAccumulator<processes=11, batch_size=64> to estimate probabilities from sliding windows
2024-05-29 13:46:03,124 - INFO - 1 batches submitted to accumulate stats from 64 documents (342328 virtual)
2024-05-29 13:46:08,969 - INFO - 11 accumulators retrieved from output queue
2024-05-29 13:46:08,991 - INFO - accumulated word occurrence stats for 342328 virtual documents


0.2759876709827538


In [13]:
# def Sort(sub_li):
#     sub_li.sort(key=lambda  x:x[1])
#     sub_li.reverse()
#     return sub_li

In [14]:
# # lda_model.save("../models/influencer_lda_model.model")
# new_model=gensim.models.ldamodel.LdaModel.load("../models/influencer_lda_model.model")


In [15]:
# test_doc=corpus[-1]
# vector=new_model[test_doc]
# sorted_vector=Sort(vector)
# print(sorted_vector)